In [ ]:
from datasets import load_dataset, get_dataset_config_names
squad_data_train, squad_data_test = load_dataset('squad_v2', split=['train','validation'])


In [ ]:
squad_data_train

In [ ]:
import pandas as pd
from operator import itemgetter
df_train = pd.DataFrame(columns = ['context','question','answer'])
df_train['context'], df_train['question'], df_train['answer'] = squad_data_train['context'],  squad_data_train['question'],  list(map(itemgetter('text'), squad_data_train['answers'] ))
df_train

In [ ]:
df_test= pd.DataFrame(columns = ['context','question','answer'])
df_test['context'], df_test['question'], df_test['answer'] = squad_data_test['context'],  squad_data_test['question'],  list(map(itemgetter('text'), squad_data_test['answers'] ))

df_train.append(df_test, ignore_index = True)


In [ ]:
# removing rows with no answers
df_train = df_train.loc[df_train['answer'].apply(lambda x: len(x) == 1)]
# making list of answer to a string
df_train['answer'] = df_train['answer'].apply(lambda x: x[0])
df_train.reset_index(drop=True)

In [ ]:
import re
df_train['answer'] = df_train['answer'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df_train['context'] = df_train['context'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df_train['question'] = df_train['question'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df_train

In [ ]:
# removing rows with long answer
df_train  = df_train.loc[df_train['answer'].apply(lambda x: len(x.split()) <= 6)]
df_train.reset_index(drop=True)

In [ ]:
#make all lower case
df_train['context'], df_train['question'], df_train['answer'] = df_train['context'].str.lower(), df_train['question'].str.lower(), df_train['answer'].str.lower()
df_train = df_train.reset_index(drop=True)
df_train

In [ ]:
# add <bos> and <eos> symbol for answer
df_train['answer'] = df_train['answer'].apply(lambda x: 'bos ' + x + ' eos' )
df_train

In [ ]:
from nltk import word_tokenize
df_train['answer'] = df_train.apply(lambda row: word_tokenize(row['answer']), axis=1)
df_train['question'] = df_train.apply(lambda row: word_tokenize(row['question']), axis=1)
df_train['context'] = df_train.apply(lambda row: word_tokenize(row['context']), axis=1)
df_train

In [ ]:
import pandas as pd
#df_train.to_pickle("./data_tokenized2.pkl")  
df_train = pd.read_pickle("../input/squadtokenised/data_tokenized2 (1).pkl")  
df_train

In [ ]:
df_train['context'].apply(len).min()

In [ ]:
#change the values later
max_c_len = 130
max_q_len = 20
max_a_len = 6

In [ ]:
# creating numpy arrays of text
import numpy as np
train_c = np.array(df_train['context'])
train_q = np.array(df_train['question'])
train_a = np.array(df_train['answer'])
txt = np.concatenate((train_a, train_q, train_c), axis=None)


In [ ]:
#train tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer 
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(txt)

In [ ]:
voc_size = len(x_tokenizer.word_index) + 1
voc_size

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
#!unzip glove*.zip

In [ ]:
################################testing

f = open('../input/glove-embeddings/glove.6B.300d.txt', 'r')
lines = f.readlines()

from tqdm import tqdm
import numpy as np
words_list = []
glove_emb = {}
for i, line in enumerate(lines):
    line = line.strip('\n')
    tokens = line.split()
    temp = list(map(float, tokens[1:]))
    word = tokens[0].lower()
    words_list.append(word)
    glove_emb[word] = np.array(temp)
    #print(len(temp))




In [ ]:
num_tokens = len(x_tokenizer.word_index) + 1
embedding_dim = 300
hits = 0
misses = 0
m = []
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in x_tokenizer.word_index.items():
    try:
        embedding_vector = glove_emb[word]
    
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    except:
        misses += 1
        m.append(word)
        #print(word)
print("Converted %d words (%d misses)" % (hits, misses))

################################testing ended


In [ ]:
 x_tokenizer.texts_to_sequences(['bos','eos'])

In [ ]:
#embedding_matrix[11]

In [ ]:
#creating embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_emb_c = x_tokenizer.texts_to_sequences(train_c) 
train_emb_q = x_tokenizer.texts_to_sequences(train_q) 
train_emb_a = x_tokenizer.texts_to_sequences(train_a) 

In [ ]:
# making all instances same length
train_emb_c = pad_sequences(train_emb_c,  maxlen=max_c_len, padding='post')
train_emb_q = pad_sequences(train_emb_q,  maxlen=max_q_len, padding='post')
train_emb_a = pad_sequences(train_emb_a,  maxlen=max_a_len, padding='post')

In [ ]:
max_c_len


In [ ]:
#model 1 
from keras import backend as K 
K.clear_session() 
latent_dim = 300 
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Input, Dense,Embedding, Concatenate, TimeDistributed,Attention
from tensorflow.keras.models import Model,load_model, model_from_json
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import pickle as pkl
import numpy as np

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Attention
K.clear_session() 
latent_dim = 300
# Encoder 
encoder_inputs = Input(shape=(max_q_len,)) 
enc_emb = Embedding(voc_size, latent_dim,weights=[embedding_matrix],trainable=True)(encoder_inputs)
#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)
#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output, state_h, state_c = encoder_lstm2(encoder_output1)
'''

#LSTM 3 
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)
'''

#context lstm
 
context_inputs = Input(shape=(max_c_len,)) 
contxt_emb = Embedding(voc_size, latent_dim, weights=[embedding_matrix],trainable=True)(context_inputs) 
#LSTM 1 
context_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True) 
context_output1, context_state_h1, context_state_c1 = context_lstm1(contxt_emb) 


#LSTM 2 
context_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True) 
context_output, context_state_h, context_state_c = context_lstm2(context_output1)

'''
#LSTM 3 
context_lstm3 = LSTM(latent_dim,return_sequences=True,return_state=True) 
context_output, context_state_h, context_state_c = context_lstm3(context_output2)
'''


# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(voc_size, latent_dim,weights=[embedding_matrix],trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs)
#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#Attention Layer
attn_out = Attention()([decoder_outputs , context_output]) 
#attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])


# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])
#Dense layer
decoder_dense = TimeDistributed(Dense(voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input)
# Define the model
model = Model([encoder_inputs, context_inputs, decoder_inputs], decoder_outputs) 
plot_model(model, to_file='train_model.png', show_shapes=True)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split( train_emb_q, train_emb_a, train_emb_c, test_size=0.2, random_state=42)

In [ ]:
history = model.fit([X_train, c_train, y_train[:,:-1]], y_train.reshape(y_train.shape[0], y_train.shape[1])[:,1:], 
                    epochs=40, 
                    callbacks=[es],
                    batch_size=512,
                   )


In [ ]:
####model1 inference  # reference == https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

latent_dim=300
# encoder inference

#print(encoder_outputs.shape)
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_output, state_h, state_c])
context_model = Model(inputs=context_inputs,outputs=[context_output, context_state_h, context_state_c])
# decoder inference

# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

decoder_hidden_state_input = Input(shape=(max_q_len, latent_dim))
decoder_context_state_input = Input(shape=(max_c_len, latent_dim))


dec_emb2= dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

attn_out_inf = Attention()([decoder_outputs2 , decoder_context_state_input]) 

decoder_inf_concat = Concatenate(axis=-1, name='concat_layer')([decoder_outputs2, attn_out_inf])
# A dense softmax layer to generate prob dist. over the target vocabulary

decoder_outputs2 = decoder_dense(decoder_inf_concat)
# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_context_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence2(input_seq, conext_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict([input_seq])
    c_out, c_h, c_c = context_model.predict([conext_seq])
    #context_output, context_state_h, context_state_c = context_model.predict(context)
    #context_vector, attention_weights = att(e_h, context_output)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Chose the 'bos' word as the first word of the target sequence
    target_seq[0, 0] = word_to_token['bos']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out,c_out, e_h, e_c])
        # prediction
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:
            break
        else:
            sampled_token = token_to_word[sampled_token_index]
        if(sampled_token!='eos'):
              decoded_sentence += ' '+sampled_token
        # stop prediction condition
        if (sampled_token == 'eos' or len(decoded_sentence.split()) >= (max_a_len)):
                  stop_condition = True
        # Update the target sequence 
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        # Update internal states
        e_h, e_c = h, c
    return decoded_sentence

In [ ]:
def seq2text(input_seq):
    newString = ''
    for i in input_seq:
        if i != 0:
            newString = newString + token_to_word[i] + ' '

    return newString

In [ ]:
#seq2text(train_emb_q[6]),seq2text(train_emb_q[5])

In [ ]:
word_to_token = x_tokenizer.word_index
token_to_word = x_tokenizer.index_word

In [ ]:
print('predicted')
print(decode_sequence2(train_emb_q[102].reshape(1, max_q_len),train_emb_c[102].reshape(1, max_c_len)))
print('actual')
print(seq2text(train_emb_a[102]),seq2text(train_emb_q[102]))

In [ ]:
print('predicted')
print(decode_sequence2(X_test[102].reshape(1, max_q_len),c_test[102].reshape(1, max_c_len)))
print('actual')
print(seq2text(y_test[102]),seq2text(X_test[102]))
 

In [ ]:
def lcs(X, Y, m, n):
 
  
    if m == 0 or n == 0:
       return 0;
    elif X[m-1] == Y[n-1]:
       return 1 + lcs(X, Y, m-1, n-1);
    else:
       return max(lcs(X, Y, m, n-1), lcs(X, Y, m-1, n));

In [ ]:
def decode_accuracy2(input_seq, conext_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict([input_seq])
    c_out, c_h, c_c = context_model.predict([conext_seq])
    #context_output, context_state_h, context_state_c = context_model.predict(context)
    #context_vector, attention_weights = att(e_h, context_output)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Chose the 'bos' word as the first word of the target sequence
    target_seq[0, 0] = word_to_token['bos']
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out,c_out, e_h, e_c])
        # prediction
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:
            break
        else:
            sampled_token = token_to_word[sampled_token_index]
        if(sampled_token!='eos'):
              decoded_sentence.append(sampled_token_index)
        # stop prediction condition
        if (sampled_token == 'eos' or len(decoded_sentence) >= (max_a_len)):
                  stop_condition = True
        # Update the target sequence 
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        # Update internal states
        e_h, e_c = h, c
    return decoded_sentence

In [ ]:
# calculation of precision recall
#X_train, X_test, y_train, y_test, c_train, c_test
#Precision = #common idxs / #predicted idxs
#Recall = #common idxs / #ground truth idxs
rec = 0
prec = 0
from tqdm import tqdm
for i in tqdm(range(len(X_test[1:1000]))):

    x=X_test[i]
    y=y_test[i]
    c=c_test[i]
    pred = decode_accuracy2(x.reshape(1, max_q_len),c.reshape(1, max_c_len))
    lcs_score = lcs(y, pred, len(y), len(pred))
    if(len(pred) != 0):
        prec += (lcs_score/len(pred))
    else:
        prec += (lcs_score/1)
    rec += (lcs_score/len(y))
print('precision : ', (prec/len(X_test)))
print('recall : ', (rec/len(X_test)))
prec = prec/len(X_test))
rec = rec/len(X_test))
f_meaure =  (2*prec*rec)/(prec+rec)
print('f_meaure :',f_meaure)

In [ ]:
prec = prec/len(X_test)
rec = rec/len(X_test)
f_meaure =  (2*prec*rec)/(prec+rec)
print('f_meaure :',f_meaure)

In [ ]:
# calculation of precision recall
#X_train, X_test, y_train, y_test, c_train, c_test
#Precision = #common idxs / #predicted idxs
#Recall = #common idxs / #ground truth idxs
rec = 0
prec = 0
from tqdm import tqdm
for i in tqdm(range(len(X_test[1:500]))):
    x=X_test[i]
    if(x[0]!=):
        continue
    y=y_test[i]
    c=c_test[i]
    pred = decode_accuracy2(x.reshape(1, max_q_len),c.reshape(1, max_c_len))
    lcs_score = lcs(y, pred, len(y), len(pred))
    if(len(pred) != 0):
        prec += (lcs_score/len(pred))
    else:
        prec += (lcs_score/1)
    rec += (lcs_score/len(y))
print('precision : ', (prec/len(X_test)))
print('recall : ', (rec/len(X_test)))
f_meaure =  (2*prec*rec)/(prec+rec)
print('f_meaure :',f_meaure)


In [ ]:
from nltk import word_tokenize
import numpy as np
def demo(context, question):
    q = re.sub(r'[^\w\s]', '', question_d.lower())
    c = re.sub(r'[^\w\s]', '', context_d.lower())
    q_d = word_tokenize(q)
    q_d = [q_d]
    c_d = word_tokenize(c)
    c_d = [c_d]
    q_d_t = np.array(x_tokenizer.texts_to_sequences(q_d))
    c_d_t = np.array(x_tokenizer.texts_to_sequences(c_d))
    q_d_p = pad_sequences(q_d_t,  maxlen=max_q_len, padding='post')
    c_d_p = pad_sequences(c_d_t,  maxlen=max_c_len, padding='post')
    print(decode_sequence2(q_d_p.reshape(1, max_q_len),c_d_p.reshape(1, max_c_len)))
    #print(q_d_p.shape)
    

In [ ]:
context_d = "Another important library – the University Library, founded in 1816, is home to over two million items. The building was designed by architects Marek Budzyński and Zbigniew Badowski and opened on 15 December 1999. It is surrounded by green. The University Library garden, designed by Irena Bajerska, was opened on 12 June 2002. It is one of the largest and most beautiful roof gardens in Europe with an area of more than 10,000 m2 (107,639.10 sq ft), and plants covering 5,111 m2 (55,014.35 sq ft). As the university garden it is open to the public every day."

In [ ]:
question_d = "Who designed the garden for the University Library?"

In [ ]:
demo(context_d, question_d)
#train_q[0:2],q_d